# Gold

## Criar schema

In [0]:
create schema if not exists juntos_somos_mais.gold

## Criar tabela

## Tabela: Order

In [0]:
-- Objetivo: representar uma linha por pedido (isto é, foco na transação como unidade)
create or replace temp view new_orders as
select 
  date, 
  transaction_id, 
  customer_id,
  count(distinct product_id) distinct_products, 
  sum(quantity) total_products, 
  round(sum(price)/sum(quantity),2) avg_ticket, 
  round(sum(price * quantity),2) total_transaction, 
  transaction_status
from juntos_somos_mais.silver.transactions_cleaned
group by date, transaction_id, customer_id, transaction_status

In [0]:
select * from new_orders
order by date, transaction_id
limit 10

date,transaction_id,customer_id,distinct_products,total_products,avg_ticket,total_transaction,transaction_status
2018-12-01,536365,17850,7,40,2.49,552.52,not cancelled
2018-12-01,536366,17850,2,12,2.03,145.8,not cancelled
2018-12-01,536367,13047,12,83,2.2,1136.34,not cancelled
2018-12-01,536368,13047,4,15,4.04,225.54,not cancelled
2018-12-01,536369,13047,1,3,5.45,49.05,not cancelled
2018-12-01,536370,12583,19,446,0.52,5393.02,not cancelled
2018-12-01,536371,13748,1,80,0.16,1028.8,not cancelled
2018-12-01,536372,17850,2,12,2.03,145.8,not cancelled
2018-12-01,536373,17850,16,88,2.48,1168.28,not cancelled
2018-12-01,536374,15100,1,32,0.67,687.04,not cancelled


In [0]:
create table if not exists juntos_somos_mais.gold.orders (
  date DATE,
  transaction_id INT,
  customer_id BIGINT,
  distinct_products INT,
  total_products INT,
  avg_ticket DOUBLE,
  total_transaction DOUBLE,
  transaction_status STRING
)
using delta
comment "Gold layer - aggregated orders per transaction"; 

merge into juntos_somos_mais.gold.orders as target
using new_orders as source
ON target.transaction_id = source.transaction_id
when matched then update set
  target.transaction_status = source.transaction_status
when not matched then 
insert *;

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
23204,0,0,23204


In [0]:
select * from juntos_somos_mais.gold.orders
limit 10

date,transaction_id,customer_id,distinct_products,total_products,avg_ticket,total_transaction,transaction_status
2019-07-18,560393,14808,49,465,1.38,5241.95,not cancelled
2019-07-18,560434,15434,442,1192,4.93,14543.33,not cancelled
2019-07-18,560368,17841,113,384,3.84,4556.34,not cancelled
2019-07-18,560291,15291,44,70,9.92,1062.19,not cancelled
2019-07-18,560394,14390,39,310,1.47,3294.97,not cancelled
2019-07-18,560437,13940,15,100,2.14,1248.97,not cancelled
2019-07-18,560290,12290,31,49,8.7,642.38,not cancelled
2019-07-18,560385,14583,60,145,5.53,1779.05,not cancelled
2019-07-18,560433,15433,236,580,5.44,7237.87,not cancelled
2019-07-18,560287,13821,17,43,4.77,484.79,not cancelled


### Tabela: Product item

In [0]:
create or replace temp view new_product_items as
select 
  date, 
  transaction_id, 
  customer_id,
  product_id, 
  product_name,  
  price,
  quantity,
  round(price * quantity,2) item_total
from juntos_somos_mais.silver.transactions_cleaned; 

create table if not exists juntos_somos_mais.gold.product_items (
  date DATE,
  transaction_id INT,
  customer_id BIGINT,
  product_id STRING,
  product_name STRING,
  price DOUBLE,
  quantity INT, 
  item_total DOUBLE
)
using delta
comment "Gold layer - individual product items per transaction"; 

merge into juntos_somos_mais.gold.product_items as target
using new_product_items as source
on target.transaction_id = source.transaction_id
and target.product_id = source.product_id
and target.date = source.date
when not matched then
insert 
( 
  date, 
  transaction_id,
  customer_id, 
  product_id, 
  product_name, 
  price, 
  quantity, 
  item_total) 
values (
  source.date, 
  source.transaction_id,
  source.customer_id, 
  source.product_id, 
  source.product_name, 
  source.price, 
  source.quantity, 
  source.item_total);

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
531150,0,0,531150


In [0]:
select * from juntos_somos_mais.gold.product_items
limit 10

date,transaction_id,customer_id,product_id,product_name,price,quantity,item_total
2019-11-16,576840,14840,71459,Hanging Jam Jar T-Light Holders,11.95,4,47.8
2019-11-15,576538,14725,72800B,4 Purple Flock Dinner Candles,12.86,4,51.44
2019-11-15,576570,17220,22900,Set 2 Tea Towels I Love London,13.58,6,81.48
2019-11-15,576618,17618,22494,Emergency First Aid Tin,12.77,2,25.54
2019-11-14,576093,17744,21162,Toxic Area Door Hanger,10.65,24,255.6
2019-11-14,576254,16422,20677,Pink Polkadot Bowl,11.34,72,816.48
2019-11-14,576321,14953,21730,Glass Star Frosted T-Light Holder,15.32,4,61.28
2019-11-14,576329,15329,20837,French Floral Cushion Cover,13.62,3,40.86
2019-11-14,576226,13268,22591,Cardholder Gingham Christmas Tree,13.58,1,13.58
2019-11-13,575935,17838,21876,Pottering Mug,11.94,12,143.28
